In [ ]:
!pip -q install kaggle
import zipfile, pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración básica
sns.set()
plt.rcParams['figure.figsize'] = (10, 6)

pathlib.Path('/root/.kaggle').mkdir(parents=True, exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
# Descarga
!kaggle datasets download -d blastchar/telco-customer-churn -p /content

# Descomprimir
with zipfile.ZipFile('/content/telco-customer-churn.zip', 'r') as z:
    z.extractall('/content/telco-customer-churn')

!ls -lh /content/telco-customer-churn

churn_df = pd.read_csv('/content/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')


In [ ]:
# Análisis exploratorio enfocado en demografía
print("ANALISIS DEMOGRAFICO")

# Información general
print(f"\nInformación del dataset:")
print(f"Total de clientes: {len(churn_df):,}")
print(f"Tasa de churn general: {(churn_df['Churn'] == 'Yes').mean()*100:.1f}%")

# Análisis por género
print(f"\nAnálisis por Género:")
gender_churn = churn_df.groupby('gender')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).round(1)
for gender, rate in gender_churn.items():
    count = len(churn_df[churn_df['gender'] == gender])
    print(f"  {gender}: {rate}% churn ({count:,} clientes)")

# Análisis por ciudadanos mayores
print(f"\nAnálisis por Edad (Senior Citizen):")
senior_churn = churn_df.groupby('SeniorCitizen')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).round(1)
senior_labels = {0: 'No Senior', 1: 'Senior'}
for senior, rate in senior_churn.items():
    count = len(churn_df[churn_df['SeniorCitizen'] == senior])
    label = senior_labels[senior]
    print(f"  {label}: {rate}% churn ({count:,} clientes)")

# Análisis por dependientes
print(f"\nAnálisis por Dependientes:")
dep_churn = churn_df.groupby('Dependents')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).round(1)
for dep, rate in dep_churn.items():
    count = len(churn_df[churn_df['Dependents'] == dep])
    print(f"  {dep}: {rate}% churn ({count:,} clientes)")


In [ ]:
# Visualizaciones demográficas
print("VISUALIZACIONES DEMOGRAFICAS")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gráfico 1: Churn por género
gender_data = churn_df.groupby(['gender', 'Churn']).size().unstack()
gender_data.plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Distribución de Churn por Género')
axes[0,0].set_xlabel('Género')
axes[0,0].set_ylabel('Número de Clientes')
axes[0,0].legend(['No Churn', 'Churn'])
axes[0,0].tick_params(axis='x', rotation=0)

# Gráfico 2: Churn por Senior Citizen
senior_data = churn_df.copy()
senior_data['SeniorCitizen'] = senior_data['SeniorCitizen'].map({0: 'No Senior', 1: 'Senior'})
sns.countplot(data=senior_data, x='SeniorCitizen', hue='Churn', ax=axes[0,1])
axes[0,1].set_title('Distribución de Churn por Edad')
axes[0,1].set_xlabel('Tipo de Cliente')
axes[0,1].set_ylabel('Número de Clientes')

# Gráfico 3: Churn por dependientes
sns.countplot(data=churn_df, x='Dependents', hue='Churn', ax=axes[1,0])
axes[1,0].set_title('Distribución de Churn por Dependientes')
axes[1,0].set_xlabel('Tiene Dependientes')
axes[1,0].set_ylabel('Número de Clientes')

# Gráfico 4: Tenure vs Churn
sns.boxplot(data=churn_df, x='Churn', y='tenure', ax=axes[1,1])
axes[1,1].set_title('Distribución de Tenure por Churn')
axes[1,1].set_xlabel('Churn')
axes[1,1].set_ylabel('Tenure (meses)')

plt.tight_layout()
plt.show()


In [ ]:
# Análisis de perfiles de riesgo
print("PERFILES DE RIESGO DE CHURN")

# Crear perfil de riesgo basado en características demográficas
def calcular_riesgo(row):
    score = 0
    
    # Senior citizens tienen mayor riesgo
    if row['SeniorCitizen'] == 1:
        score += 1
    
    # Clientes sin dependientes tienen mayor riesgo
    if row['Dependents'] == 'No':
        score += 1
    
    # Clientes sin pareja tienen mayor riesgo
    if row['Partner'] == 'No':
        score += 1
    
    # Tenure bajo indica mayor riesgo
    if row['tenure'] < 12:
        score += 1
    
    return score

churn_df['risk_score'] = churn_df.apply(calcular_riesgo, axis=1)

# Definir categorías de riesgo
def categorizar_riesgo(score):
    if score == 0:
        return 'Bajo'
    elif score <= 2:
        return 'Medio'
    else:
        return 'Alto'

churn_df['risk_category'] = churn_df['risk_score'].apply(categorizar_riesgo)

# Análisis de riesgo vs churn real
risk_analysis = churn_df.groupby('risk_category')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).round(1)

print("\nTasa de churn por categoría de riesgo:")
for category, rate in risk_analysis.items():
    count = len(churn_df[churn_df['risk_category'] == category])
    print(f"  Riesgo {category}: {rate}% churn ({count:,} clientes)")

# Visualización del análisis de riesgo
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.countplot(data=churn_df, x='risk_category', hue='Churn', order=['Bajo', 'Medio', 'Alto'])
plt.title('Distribución de Churn por Categoría de Riesgo')
plt.xlabel('Categoría de Riesgo')
plt.ylabel('Número de Clientes')

plt.subplot(1, 2, 2)
risk_rates = churn_df.groupby('risk_category')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100)
risk_rates.reindex(['Bajo', 'Medio', 'Alto']).plot(kind='bar', color=['green', 'orange', 'red'])
plt.title('Tasa de Churn por Categoría de Riesgo')
plt.xlabel('Categoría de Riesgo')
plt.ylabel('Tasa de Churn (%)')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

print(f"\nConclusión: Los clientes de riesgo alto tienen {risk_analysis['Alto']:.1f}% de churn vs {risk_analysis['Bajo']:.1f}% en riesgo bajo")
